# Determine acceleration with Kalman filter for all tests

# Purpose
Yaw rate, Yaw acceleration and Yaw jerk are missing from the model test. These signals can be estimated with Kalman filters. This will be done for all runs in this project.

# Methodology
Loop through all tests and determine the "hidden states" with Kalman filters.

# Setup

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp
import seaborn as sns

## Local packages:
from vessel_manoeuvring_models.data import mdl




In [ ]:
from vessel_manoeuvring_models.data import kalman_filter

In [ ]:
id=22606
df, units, meta_data = mdl.load(id=id)

In [ ]:
df.head()

In [ ]:
df1 = kalman_filter.yaw(df=df, observation_covariance=0.5)
df2 = kalman_filter.yaw(df=df, observation_covariance=1000)
df3 = kalman_filter.yaw(df=df, observation_covariance=1000000)


In [ ]:
epsilon1 = df1['psi']-df1['psi_filtered']
epsilon2 = df2['psi']-df2['psi_filtered']
epsilon3 = df3['psi']-df3['psi_filtered']

fig,ax=plt.subplots()
ax.plot(df.index,  epsilon1  ,label=r'$\epsilon$', alpha=0.5, zorder=3)
ax.plot(df.index,  epsilon2 ,label=r'$\epsilon2$', alpha=0.5, zorder=2)
ax.plot(df.index,  epsilon3 ,label=r'$\epsilon3$', alpha=0.5, zorder=1)
ax.legend()

sns.displot(epsilon1)
sns.displot(epsilon2)
sns.displot(epsilon3)

In [ ]:
import scipy.stats as stats
fig,ax=plt.subplots()
stats.probplot(epsilon1, dist="norm", plot=ax);

fig,ax=plt.subplots()
stats.probplot(epsilon2, dist="norm", plot=ax);

fig,ax=plt.subplots()
stats.probplot(epsilon3, dist="norm", plot=ax);



In [ ]:
df = kalman_filter.filter_and_transform(df=df)

In [ ]:
fig,ax=plt.subplots()
df.plot(y='u', ax=ax)
df.plot(y='u_gradient', alpha=0.5, ax=ax)

fig,ax=plt.subplots()
df.plot(y='u1d', ax=ax)
df.plot(y='u1d_gradient', alpha=0.5, ax=ax)

fig,ax=plt.subplots()
df.plot(y='u1d', alpha=0.5, ax=ax)

In [ ]:
df_runs = pd.read_csv('../data/raw/runs.csv', index_col=0)

In [ ]:
save_dir = '../data/processed/kalman'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for id, row in df_runs.iterrows():
    
    print(f'working on:{id}')
    df = mdl.load_run(id=id)
    df = kalman_filter.filter_and_transform(df=df)
    
    df.index = df.index.total_seconds()
    save_name = f'{id}.csv'
    save_path = os.path.join(save_dir,save_name)
    df.to_csv(save_path)
    

In [ ]:
mask = ((~df_runs['sailing']) &
        (
            (df_runs['test_type'] == 'reference speed') |
            (df_runs['test_type'] == 'rodergrundvinkel') |
            ((df_runs['series_number'] == 5) & (df_runs['test_number'] == 1)  & (df_runs['run_number'] == 3)) |
            ((df_runs['series_number'] == 5) & (df_runs['test_number'] == 2)  & (df_runs['run_number'] == 6)) |
            ((df_runs['series_number'] == 5) & (df_runs['test_number'] == 3)  & (df_runs['run_number'] == 2)) |
            ((df_runs['series_number'] == 5) & (df_runs['test_number'] == 4)  & (df_runs['run_number'] == 1)) |
            ((df_runs['series_number'] == 5) & (df_runs['test_number'] == 5)  & (df_runs['run_number'] == 1)) 
            
       ))
df_runs_selected = df_runs.loc[mask].copy()